<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D9%87%D9%8A%D8%AF%D8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة بنفس الاتجاه."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز بنفس الاتجاه
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": position.type,  # نفس نوع الصفقة (0 = BUY, 1 = SELL)
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position in the same direction",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")


In [ ]:
#هيدج طبيعي
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز معكوس إذا كان العدد أقل من 7
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": 1 if position.type == 0 else 0,  # 1 = SELL, 0 = BUY
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار من المفكرة الاقتصادية لموقع Investing."""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    high_impact_news = []

    # استخراج الأخبار ذات التأثير العالي على الدولار
    for row in soup.find_all('tr'):
        impact = row.find('td', class_='impact')
        if impact and 'high' in impact['class']:
            time = row.find('td', class_='date').text.strip()
            currency = row.find('td', class_='currency').text.strip()
            if currency == 'USD':  # التأكد من أن الخبر متعلق بالدولار
                news_time = datetime.strptime(time, "%d %b %Y %H:%M")
                high_impact_news.append({"time": news_time.strftime("%Y-%m-%d %H:%M:%S"), "impact": "high"})

    return high_impact_news

def is_within_news_timeframe():
    """تحقق مما إذا كان الوقت الحالي ضمن فترة الأخبار."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            return True  # الوقت الحالي ضمن فترة الأخبار
    return False

def close_profitable_positions_before_after_news(symbol):
    """إغلاق الصفقات الرابحة قبل وبعد صدور الأخبار ذات التأثير العالي."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            positions = mt5.positions_get(symbol=symbol)
            for position in positions:
                if position.profit > 0:  # إذا كانت الصفقة رابحة
                    request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": position.ticket,
                        "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                        "volume": position.volume,
                        "price": mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask,
                        "deviation": 10,
                        "magic": 234000,
                        "comment": "Closing profitable position before/after high impact news",
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"Closed profitable position for {position.ticket}.")

def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    if is_within_news_timeframe():
        print("لا يمكن فتح صفقات خلال فترة الأخبار ذات التأثير العالي.")
        return  # لا تفتح الصفقة إذا كان الوقت ضمن فترة الأخبار

    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

# يمكنك إضافة المزيد من الوظائف أو تحسين الاستراتيجية حسب الحاجة


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import MetaTrader5 as mt5

def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار من المفكرة الاقتصادية لموقع Investing."""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    high_impact_news = []

    # استخراج الأخبار ذات التأثير العالي على الدولار
    for row in soup.find_all('tr'):
        impact = row.find('td', class_='impact')
        if impact and 'high' in impact['class']:
            time = row.find('td', class_='date').text.strip()
            currency = row.find('td', class_='currency').text.strip()
            if currency == 'USD':  # التأكد من أن الخبر متعلق بالدولار
                news_time = datetime.strptime(time, "%d %b %Y %H:%M")
                high_impact_news.append({"time": news_time.strftime("%Y-%m-%d %H:%M:%S"), "impact": "high"})

    return high_impact_news

def is_within_news_timeframe():
    """تحقق مما إذا كان الوقت الحالي ضمن فترة الأخبار."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            return True  # الوقت الحالي ضمن فترة الأخبار
    return False

def close_profitable_positions_before_after_news(symbol):
    """إغلاق الصفقات الرابحة قبل وبعد صدور الأخبار ذات التأثير العالي."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            positions = mt5.positions_get(symbol=symbol)
            for position in positions:
                if position.profit > 0:  # إذا كانت الصفقة رابحة
                    request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": position.ticket,
                        "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                        "volume": position.volume,
                        "price": mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask,
                        "deviation": 10,
                        "magic": 234000,
                        "comment": "Closing profitable position before/after high impact news",
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"Closed profitable position for {position.ticket}.")

def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    if is_within_news_timeframe():
        print("لا يمكن فتح صفقات خلال فترة الأخبار ذات التأثير العالي.")
        return  # لا تفتح الصفقة إذا كان الوقت ضمن فترة الأخبار

    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

# في دالة build_strategy، أضف هذا السطر بعد حساب مناطق العرض والطلب
close_profitable_positions_before_after_news("XAUUSD")


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى سعر الدخول (بريك إيفن) للرمز المحدد مع تقديم الستوب لوز بناءً على تقدم السعر."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            # الحصول على سعر الدخول وسعر جني الأرباح
            entry_price = position.price_open
            take_profit = position.tp  # سعر جني الأرباح

            # حساب المسافة بين سعر الدخول وسعر جني الأرباح
            distance_to_tp = take_profit - entry_price

            # حساب السعر الذي يمثل 50% و75% و90% من المسافة نحو الهدف
            target_price_50 = entry_price + (0.50 * distance_to_tp)
            target_price_75 = entry_price + (0.75 * distance_to_tp)
            target_price_90 = entry_price + (0.90 * distance_to_tp)

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid  # الحصول على السعر الحالي

            # إذا كان السعر الحالي قد تجاوز 50% نحو الهدف
            if current_price >= target_price_50:
                # تعيين وقف الخسارة إلى 12% من سعر الدخول
                new_stop_loss = entry_price + (0.12 * distance_to_tp)  # 12% من المسافة نحو جني الأرباح
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position.ticket,
                    "sl": new_stop_loss,  # تعيين وقف الخسارة إلى 12%
                    "tp": take_profit,  # الحفاظ على جني الأرباح الحالي
                    "magic": 123456,  # رقم سحري فريد
                    "comment": "Set stop loss to 12% after reaching 50%",
                    "type": position.type,  # نوع الصفقة (شراء أو بيع)
                }
                result = mt5.order_send(request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Failed to adjust stop loss to 12%: {result.retcode}")

            # إذا وصل السعر إلى 75% نحو الهدف، قدم الستوب لوز إلى 63% نحو التيك بروفت
            if current_price >= target_price_75:
                new_stop_loss = take_profit - (0.37 * distance_to_tp)  # 63% من المسافة نحو التيك بروفت
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position.ticket,
                    "sl": new_stop_loss,
                    "tp": take_profit,
                    "magic": 123456,
                    "comment": "Adjust stop loss to 63% towards take profit",
                    "type": position.type,
                }
                result = mt5.order_send(request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Failed to adjust stop loss to 63%: {result.retcode}")

            # إذا وصل السعر إلى 90% نحو الهدف، قدم الستوب إلى 75% نحو الستوب
            if current_price >= target_price_90:
                new_stop_loss = entry_price + (0.75 * distance_to_tp)  # 75% من المسافة نحو الستوب
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position.ticket,
                    "sl": new_stop_loss,
                    "tp": take_profit,
                    "magic": 123456,
                    "comment": "Adjust stop loss to 75% towards stop",
                    "type": position.type,
                }
                result = mt5.order_send(request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Failed to adjust stop loss to 75%: {result.retcode}")


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            # الحصول على سعر الدخول وسعر جني الأرباح
            entry_price = position.price_open
            take_profit = position.tp  # سعر جني الأرباح

            # حساب المسافة بين سعر الدخول وسعر جني الأرباح
            distance_to_tp = take_profit - entry_price

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid  # الحصول على السعر الحالي

            # حساب النسب المئوية
            fifty_percent_target = entry_price + (0.50 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.75 * distance_to_tp)
            ninety_percent_target = entry_price + (0.90 * distance_to_tp)

            # تعيين وقف الخسارة بناءً على تقدم السعر
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.63 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.75 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة لمركز معين."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,  # تعيين وقف الخسارة الجديد
        "tp": position.tp,  # الحفاظ على جني الأرباح الحالي
        "magic": 123456,  # رقم سحري فريد
        "comment": "Update stop loss",
        "type": position.type,  # نوع الصفقة (شراء أو بيع)
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to update stop loss: {result.retcode}")


In [ ]:
def scalping_hedge_strategy(symbol, volume):
    """استراتيجية الهيدج للسكالبينغ مع جني الأرباح قريبًا من وقف الخسارة بنسبة 75%."""

    # الحصول على الصفقات المفتوحة
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 2:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (2). لا يمكن فتح صفقة جديدة.")
        return

    # فتح مركز شراء
    buy_price = mt5.symbol_info_tick(symbol).ask
    stop_loss = buy_price - 5 * mt5.symbol_info(symbol).point  # وقف خسارة قريب
    take_profit = buy_price + 0.75 * (buy_price - stop_loss)  # جني أرباح بنسبة 75%

    buy_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": 0,  # 0 = BUY
        "price": buy_price,
        "sl": stop_loss,
        "tp": take_profit,
        "magic": 123456,
        "comment": "Initial Buy Position",
    }
    buy_result = mt5.order_send(buy_request)

    if buy_result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"فشل في فتح مركز الشراء: {buy_result.retcode}")
        return

    print("تم فتح مركز شراء بنجاح.")

    # تحديد مستوى الهيدج بناءً على حركة السعر
    hedge_price = mt5.symbol_info_tick(symbol).bid - 5 * mt5.symbol_info(symbol).point

    # فتح مركز بيع عند مستوى الهيدج
    sell_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": 1,  # 1 = SELL
        "price": hedge_price,
        "sl": hedge_price + 5 * mt5.symbol_info(symbol).point,  # وقف خسارة قريب
        "tp": hedge_price - 0.75 * (hedge_price - stop_loss),  # جني أرباح بنسبة 75%
        "magic": 123456,
        "comment": "Hedge Position",
    }
    sell_result = mt5.order_send(sell_request)

    if sell_result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"فشل في فتح مركز البيع: {sell_result.retcode}")
        return

    print("تم فتح مركز هيدج بنجاح.")


In [ ]:
import MetaTrader5 as mt5

def calculate_dynamic_volume(account_balance, risk_percentage):
    """حساب حجم الصفقة الديناميكي بناءً على رصيد الحساب ونسبة المخاطرة."""
    # حجم الصفقة الديناميكي
    volume = (account_balance * risk_percentage) / 1000  # تعديل حسب الحاجة
    return max(volume, 0.01)  # تأكد من أن الحجم لا يقل عن الحد الأدنى

def hedge_trade(symbol, risk_percentage=1.0):
    """فتح مركز هيدج (شراء وبيع) للرمز المحدد بحجم ديناميكي."""

    # التأكد من الاتصال بـ MetaTrader 5
    if not mt5.initialize():
        print("فشل في الاتصال بـ MetaTrader 5")
        return

    # الحصول على معلومات الحساب
    account_info = mt5.account_info()
    if account_info is None:
        print("فشل في استرداد معلومات الحساب.")
        return

    account_balance = account_info.balance
    volume = calculate_dynamic_volume(account_balance, risk_percentage)

    # الحصول على سعر الشراء الحالي
    buy_price = mt5.symbol_info_tick(symbol).ask
    sell_price = mt5.symbol_info_tick(symbol).bid

    # إعداد طلب الشراء
    buy_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_BUY,  # نوع الصفقة: شراء
        "price": buy_price,
        "sl": 0,  # وقف الخسارة (يمكن تعيينه لاحقًا)
        "tp": 0,  # جني الأرباح (يمكن تعيينه لاحقًا)
        "magic": 123456,
        "comment": "Hedge Buy Position",
    }

    # إرسال طلب الشراء
    buy_result = mt5.order_send(buy_request)

    if buy_result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"فشل في فتح مركز الشراء: {buy_result.retcode}, Error: {mt5.last_error()}")
        return

    print("تم فتح مركز شراء بنجاح.")

    # إعداد طلب البيع
    sell_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_SELL,  # نوع الصفقة: بيع
        "price": sell_price,
        "sl": 0,  # وقف الخسارة (يمكن تعيينه لاحقًا)
        "tp": 0,  # جني الأرباح (يمكن تعيينه لاحقًا)
        "magic": 123456,
        "comment": "Hedge Sell Position",
    }

    # إرسال طلب البيع
    sell_result = mt5.order_send(sell_request)

    if sell_result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"فشل في فتح مركز البيع: {sell_result.retcode}, Error: {mt5.last_error()}")
        return

    print("تم فتح مركز بيع بنجاح.")

# مثال على كيفية استخدام الدالة
hedge_trade("XAUUSD", risk_percentage=1.0)  # يمكنك استبدال "XAUUSD" بالرمز الذي تريده


In [ ]:
def dynamic_hedge(symbol, risk_percentage=0.01):
    """تطبيق استراتيجية الهيدج مع حجم ديناميكي."""
    try:
        # الحصول على معلومات الحساب
        account_info = mt5.account_info()
        if account_info is None:
            print("فشل في استرداد معلومات الحساب.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        # حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة
        volume = calculate_lot_size(account_balance, risk_percentage)

        # الحصول على الصفقات المفتوحة
        positions = mt5.positions_get(symbol=symbol)
        if not positions:
            print("لا توجد صفقات مفتوحة للرمز المحدد.")
            return

        for position in positions:
            # تحديد نوع الصفقة المعكوسة
            hedge_order_type = 1 if position.type == 0 else 0  # 1 = SELL, 0 = BUY

            # الحصول على السعر الحالي
            price = mt5.symbol_info_tick(symbol).ask if hedge_order_type == 0 else mt5.symbol_info_tick(symbol).bid

            # إعداد الطلب
            hedge_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": hedge_order_type,
                "price": price,
                "sl": 0,
                "tp": 0,
                "magic": 123456,
                "comment": "Dynamic hedge position",
            }

            # إرسال الطلب
            result = mt5.order_send(hedge_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"فشل في فتح صفقة الهيدج: {result.retcode}")
            else:
                print(f"تم فتح صفقة هيدج: {hedge_order_type} {volume} lots of {symbol} at {price}")

    except Exception as e:
        print(f"حدث خطأ أثناء تنفيذ دالة الهيدج الديناميكية: {e}")
